In [1]:
import os
import time
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import itertools
from collections import Counter
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

In [2]:

# Parameters
# =================================================

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")
# tf.flags.DEFINE_string("tweets","./data/text_emotion.csv", "Data source from crowdflower")
tf.flags.DEFINE_string("isear","./data/isear.csv", "Data source from ISEAR")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.1
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
ISEAR=./data/isear.csv
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=128
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos
TWEETS=./data/text_emotion.csv



In [3]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [4]:
data = pd.read_csv(FLAGS.isear,sep='|')

In [5]:
data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,STATE,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,11001,1,1,1,1,33,1,2,6,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11001,1,1,1,1,33,1,2,6,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11001,1,1,1,1,33,1,2,6,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11001,1,1,1,1,33,1,2,6,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11001,1,1,1,1,33,1,2,6,1,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
x_text = data["SIT"]

In [7]:
data["Field1"].unique()

array(['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt'], dtype=object)

In [8]:
    data['Field1'] = data['Field1'].map({'joy':[1,0,0,0,0,0,0], 'fear':[0,1,0,0,0,0,0], 'anger':[0,0,1,0,0,0,0], 'sadness':[0,0,0,1,0,0,0], 'disgust':[0,0,0,0,1,0,0], 'shame':[0,0,0,0,0,1,0], 'guilt':[0,0,0,0,0,0,1]})

In [10]:
y = data['Field1']

In [11]:
x_text.count()

7666

In [12]:
y.count()

7666

In [13]:
x_text[0]

'During the period of falling in love, each time that we met and \xc3\xa1 especially when we had not met for a long time.'

In [14]:
x_text=[data_helpers.clean_str(sent) for sent in x_text]

In [15]:
x_text[0]

'during the period of falling in love , each time that we met and especially when we had not met for a long time'

In [16]:
y[0]

[1, 0, 0, 0, 0, 0, 0]

In [17]:
# Build vocabulary
max_document_length = max([len(x.split(" ")) for x in x_text])
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
x = np.array(list(vocab_processor.fit_transform(x_text) ))
y = np.array(list(y))

In [18]:
x

array([[  1,   2,   3, ...,   0,   0,   0],
       [ 15,  21,  22, ...,   0,   0,   0],
       [ 15,  21,  22, ...,   0,   0,   0],
       ..., 
       [ 21,  22,  38, ...,   0,   0,   0],
       [ 21,  72,  17, ...,   0,   0,   0],
       [ 21,  16, 254, ...,   0,   0,   0]])

In [19]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [20]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [21]:
x_shuffled

array([[  15,  116,  949, ...,    0,    0,    0],
       [ 116, 3968,  115, ...,    0,    0,    0],
       [  33,   22,   19, ...,    0,    0,    0],
       ..., 
       [  15,   21,   22, ...,    0,    0,    0],
       [ 165,   37,   22, ...,    0,    0,    0],
       [  95, 1012,  111, ...,    0,    0,    0]])

In [22]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]
print x_train.shape[1]

print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))

179
Vocabulary Size: 8927
Train/Dev split: 6900/766


In [23]:
y_train.shape[1]

7

In [24]:
#data helper function
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [25]:
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=128,
            filter_sizes=[3 4 5],
            num_filters=128,
            l2_reg_lambda=0.0)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 0.5
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), 64, 200)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % 100 == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % 100 == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Writing to /home/ubuntu/cnn-text-classification-tf/runs/1491237569

2017-04-03T22:09:30.417450: step 1, loss 6.31757, acc 0.171875
2017-04-03T22:09:30.612363: step 2, loss 5.52489, acc 0.140625
2017-04-03T22:09:30.811297: step 3, loss 4.40588, acc 0.09375
2017-04-03T22:09:31.006075: step 4, loss 5.29336, acc 0.125
2017-04-03T22:09:31.203784: step 5, loss 5.12952, acc 0.140625
2017-04-03T22:09:31.394841: step 6, loss 5.03943, acc 0.15625
2017-04-03T22:09:31.592158: step 7, loss 4.94861, acc 0.125
2017-04-03T22:09:31.783684: step 8, loss 4.86245, acc 0.109375
2017-04-03T22:09:31.980101: step 9, loss 4.46962, acc 0.171875
2017-04-03T22:09:32.172149: step 10, loss 5.34941, acc 0.109375
2017-04-03T22:09:32.369979: step 11, loss 4.29757, acc 0.1875
2017-04-03T22:09:32.557327: step 12, loss 4.24104, acc 0.140625
2017-04-03T22:09:32.755123: step 13, loss 4.99609, acc 0.140625
2017-04-03T22:09:32.947285: step 14, loss 3.9522, acc 0.171875
2017-04-03T22:09:33.145156: step 15, loss 4.98548, acc 0

KeyboardInterrupt: 